please add this link to drive/My Drive to test
https://drive.google.com/drive/folders/1OZYp6W-caHjbITYlodyL-9gvxoMW4t_X?usp=sharing

preparing data
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm

%cd /content/
!unrar x '/content/drive/My Drive/cv/Training_set.rar' '/content/'
!unrar x '/content/drive/My Drive/cv/Testing_set.rar' '/content/'
train_path = '/content/Training/'
test_path = '/content/Testing_set/'


!rm /content/Training/Basketball/v_shooting_24_01.mpg
!rm /content/Training/Basketball/v_shooting_16_05.mpg

In [ ]:
classes = ['Diving', 'Jumping', 'Basketball', 'Tennis', 'Walking']


In [ ]:
import cv2
import os
import numpy as np
import argparse

SMALLEST_DIM = 256
IMAGE_CROP_SIZE = 224
FRAME_RATE = 25


# sample frames at 25 frames per second
def sample_video(video_path, path_output):
    # for filename in os.listdir(video_path):
    if video_path.endswith((".mp4", ".avi", ".mpg")):
        # filename = video_path + filename
        os.system("ffmpeg -r {1} -i {0} -q:v 2 {2}/frame_%05d.jpg".format(video_path, FRAME_RATE,
                                                                          path_output))
    else:
        raise ValueError("Video path is not the name of video file (.mp4 or .avi)")


# the videos are resized preserving aspect ratio so that the smallest dimension is 256 pixels, with bilinear interpolation
def resize(img):
    # print('Original Dimensions : ', img.shape)

    original_width = int(img.shape[1])
    original_height = int(img.shape[0])

    aspect_ratio = original_width / original_height

    if original_height < original_width:
        new_height = SMALLEST_DIM
        new_width = int(new_height * aspect_ratio)
    else:
        new_width = SMALLEST_DIM
        new_height = int(original_width / aspect_ratio)

    dim = (new_width, new_height)
    # resize image
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_LINEAR)
    # print('Resized Dimensions : ', resized.shape)

    return resized


def crop_center(img, new_size):
    y, x, c = img.shape
    (cropx, cropy) = new_size
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    return img[starty:starty + cropy, startx:startx + cropx]


def rescale_pixel_values(img):
    # print('Data Type: %s' % img.dtype)
    # print('Min: %.3f, Max: %.3f' % (img.min(), img.max()))
    img = img.astype('float32')
    # normalize to the range 0:1
    # img /= 255.0
    # normalize to the range -1:1
    img = (img / 255.0) * 2 - 1
    # confirm the normalization
    # print('Min: %.3f, Max: %.3f' % (img.min(), img.max()))
    return img


# The provided .npy file thus has shape (1, num_frames, 224, 224, 3) for RGB, corresponding to a batch size of 1
def run_rgb(sorted_list_frames):
    result = np.zeros((1, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 3))
    for full_file_path in sorted_list_frames:
        img = cv2.imread(full_file_path, cv2.IMREAD_UNCHANGED)
        
        img = pre_process_rgb(img)
        new_img = np.reshape(img, (1, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 3))
        result = np.append(result, new_img, axis=0)

    result = result[1:, :, :, :]
    result = np.expand_dims(result, axis=0)
    return result


def pre_process_rgb(img):
    resized = resize(img)
    img_cropped = crop_center(resized, (IMAGE_CROP_SIZE, IMAGE_CROP_SIZE))
    img = rescale_pixel_values(img_cropped)
    return img


def read_frames(video_path):
    list_frames = []
    for file in os.listdir(video_path):
        if file.endswith(".jpg"):
            full_file_path = video_path + file
            list_frames.append(full_file_path)
    sorted_list_frames = sorted(list_frames)
    return sorted_list_frames


def run_flow(sorted_list_frames):
    sorted_list_img = []
    for frame in sorted_list_frames:
        img = cv2.imread(frame, cv2.IMREAD_UNCHANGED)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        sorted_list_img.append(img_gray)

    result = np.zeros((1, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 2))
    prev = sorted_list_img[0]
    for curr in sorted_list_img[1:]:
        flow = compute_optical_flow(prev, curr)
        flow = pre_process_flow(flow)
        prev = curr
        result = np.append(result, flow, axis=0)

    result = result[1:, :, :, :]
    result = np.expand_dims(result, axis=0)
    return result


def pre_process_flow(flow_frame):
    resized = resize(flow_frame)
    img_cropped = crop_center(resized, (IMAGE_CROP_SIZE, IMAGE_CROP_SIZE))
    new_img = np.reshape(img_cropped, (1, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 2))
    return new_img


#  Pixel values are truncated to the range [-20, 20], then rescaled between -1 and 1
def compute_optical_flow(prev, curr):
    flow = cv2.calcOpticalFlowFarneback(prev, curr, None, 0.5, 3, 15, 3, 5, 1.1, 0)
    flow_frame = np.clip(flow, -20, 20)
    flow_frame = flow_frame / 20.0
    return flow_frame

RGB
---

In [ ]:
def extract_rgb(path, no_of_frames):
    
    #print(path)
    video_frames = []
    cap = cv2.VideoCapture(path);
    
    fps = int(cap.get(cv2.CAP_PROP_FPS)) #all 29.97002997002997
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    h = w = 256
    #print(frame_count)
    
    end = frame_count-(frame_count%no_of_frames)
    stride = int(np.ceil(frame_count/no_of_frames))
    
    if stride == 0:
        stride = 1    
    frames_taken = []
    
    i = 0
    j = 0
    while i < no_of_frames:
        for frame_no in range(j, frame_count, stride):
            if i >= no_of_frames:
                break
            frames_taken.append(frame_no)
            i+=1
        j+=1

    #frames_taken.sort()
    #print(frames_taken)
    #print(len(frames_taken))

    result = np.zeros((1, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 3))
    for frame_no in frames_taken:
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
        ret, frame = cap.read()
            
        img = pre_process_rgb(frame)
        new_img = np.reshape(img, (1, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 3))
        result = np.append(result, new_img, axis=0)
    
    result = result[1:, :, :, :]
    result = np.expand_dims(result, axis=0)
    #print(result.shape)
    return result


In [ ]:
#extract_rgb(train_path+'Basketball'+'/'+'v_shooting_24_01.mpg', 125)
#extract_rgb(train_path+'Basketball'+'/'+'v_shooting_16_05.mpg', 125)

train_paths = []
for folder in os.listdir(train_path):
    for video in os.listdir(train_path+folder):
        #extract_rgb(train_path+folder+'/'+video, 125)
        train_paths.append([train_path+folder+'/'+video, classes.index(folder)])
        

In [ ]:
print(train_paths)
print(len(train_paths))
global train_index
train_index = 0

[['/content/Training/Basketball/v_shooting_20_03.mpg', 2], ['/content/Training/Basketball/v_shooting_18_03.mpg', 2], ['/content/Training/Basketball/v_shooting_06_04.mpg', 2], ['/content/Training/Basketball/v_shooting_16_07.mpg', 2], ['/content/Training/Basketball/v_shooting_03_01.mpg', 2], ['/content/Training/Basketball/v_shooting_06_02.mpg', 2], ['/content/Training/Basketball/v_shooting_18_02.mpg', 2], ['/content/Training/Basketball/v_shooting_10_03.mpg', 2], ['/content/Training/Basketball/v_shooting_19_06.mpg', 2], ['/content/Training/Basketball/v_shooting_11_02.mpg', 2], ['/content/Training/Basketball/v_shooting_19_02.mpg', 2], ['/content/Training/Basketball/v_shooting_04_02.mpg', 2], ['/content/Training/Basketball/v_shooting_01_04.mpg', 2], ['/content/Training/Basketball/v_shooting_02_01.mpg', 2], ['/content/Training/Basketball/v_shooting_10_02.mpg', 2], ['/content/Training/Basketball/v_shooting_13_04.mpg', 2], ['/content/Training/Basketball/v_shooting_16_03.mpg', 2], ['/content/Tr

flows
---

In [ ]:
def extract_flow(path, no_of_frames):
    no_of_frames+=1
    cap = cv2.VideoCapture(path);
    
    fps = int(cap.get(cv2.CAP_PROP_FPS)) #all 29.97002997002997
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    h = w = 256
    
    end = frame_count-(frame_count%no_of_frames)
    stride = int(np.ceil(frame_count/no_of_frames))
    
    if stride == 0:
        stride = 1    
    frames_taken = []
    
    i = 0
    j = 0
    while i < no_of_frames:
        for frame_no in range(j, frame_count, stride):
            if i >= no_of_frames:
                break
            frames_taken.append(frame_no)
            i+=1
        j+=1

    #frames_taken.sort()
    #print(frames_taken)
    #print(len(frames_taken))

    #cap.set(cv2.CAP_PROP_POS_FRAMES, 3)
    #print(cap.get(cv2.CAP_PROP_POS_FRAMES))


    sorted_list_img = []
    for frame_no in frames_taken:
        #print('fn: ',frame_no)      
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
        #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
        ret, frame = cap.read()
        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        sorted_list_img.append(img_gray)
    
    #print('gray done')
    result = np.zeros((1, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 2))    
    prev = sorted_list_img[0]
    for curr in sorted_list_img[1:]:
        flow = compute_optical_flow(prev, curr)
        #print('compute_optical_flow done')

        flow = pre_process_flow(flow)
        #print('pre_process_flow done')

        prev = curr
        result = np.append(result, flow, axis=0)

    result = result[1:, :, :, :]
    result = np.expand_dims(result, axis=0)
    #print(result.shape)
    return result


    

In [ ]:
for folder in os.listdir(train_path):
    for video in os.listdir(train_path+folder):
        #extract_flow(train_path+folder+'/'+video, 125)
        pass
 

MODEL
---

In [ ]:
!git clone https://github.com/dlpbc/keras-kinetics-i3d.git

Cloning into 'keras-kinetics-i3d'...
remote: Enumerating objects: 49, done.
remote: Total 49 (delta 0), reused 0 (delta 0), pack-reused 49
Unpacking objects: 100% (49/49), done.


In [ ]:
#!pip install keras-video-generators

In [ ]:
%cd /content/keras-kinetics-i3d/

from __future__ import print_function
from __future__ import absolute_import

import warnings

import numpy as np

from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import AveragePooling3D
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import Lambda
from keras.layers import GlobalAveragePooling3D

from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from i3d_inception import conv3d_bn
from keras.optimizers import Adam


/content/keras-kinetics-i3d


Using TensorFlow backend.


train rgb model(takes from 3 to 5 hrs depending on #input_frames)
---

In [25]:
import numpy as np
import argparse

from i3d_inception import Inception_Inflated3d

NUM_CLASSES = 5
FRAME_HEIGHT = FRAME_WIDTH = 224
NUM_RGB_CHANNELS = 3
NUM_FLOW_CHANNELS = 2
NUM_FRAMES = 64

In [ ]:
#define models
# build model for rgb data
# and load pretrained weights (trained on imagenet and kinetics dataset)
rgb_model = Inception_Inflated3d(
                include_top=False,
                weights='rgb_imagenet_and_kinetics',
                input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS),
                classes=NUM_CLASSES)
 

In [ ]:
#rgb_model.summary()

In [ ]:
x = rgb_model.get_layer('global_avg_pool').output

x = Dropout(0.0)(x)

x = conv3d_bn(x, NUM_CLASSES, 1, 1, 1, padding='same', 
        use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')
 
num_frames_remaining = int(x.shape[1])
x = Reshape((num_frames_remaining, NUM_CLASSES))(x)

# logits (raw scores for each class)
x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
            output_shape=lambda s: (s[0], s[2]))(x)

x = Activation('softmax', name='prediction')(x)

F_rgb_model = Model(rgb_model.get_layer(index = 0).output, x, name='i3d_inception_rgb')

In [ ]:
F_rgb_model.summary()

In [ ]:
from random import shuffle
shuffle(train_paths)
train = train_paths[0:]
val = train_paths[337:]
print(len(train))
print(len(val))

global train_index
train_index = 0
global test_index
test_index = 0

472
135


In [ ]:


def load_data(batch_size, paths, train_bool):
    
    index = 0
    if train_bool:
        global train_index
        if train_index == len(paths):
            train_index = 0
        index = train_index
    else:
        global test_index
        if test_index == len(paths):
            test_index = 0
        index = test_index

    example = paths[index]
    f = extract_rgb(example[0], NUM_FRAMES)
    
    if train_bool:
        train_index+=1
    else:
        test_index+=1
    
    return (f, [example[1]])

def batch_generator(batch_size, steps, paths, train_bool):
    
    idx=1
    while True: 
        yield load_data(batch_size, paths, train_bool)## Yields data
        if idx<steps:
            idx+=1
        else:
            idx=1
train_batch_generator = batch_generator(1, 1, train, True)
val_batch_generator = batch_generator(1, 1, val, False)

In [ ]:
F_rgb_model.compile(optimizer=Adam(0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#F_rgb_model.load_weights('/content/drive/My Drive/cv/a_rgb_model.h5')

In [ ]:
#F_rgb_model.fit_generator(train_batch_generator, epochs=100, steps_per_epoch=377//1, verbose=1, validation_data=val_batch_generator, validation_steps=135//1) #train_length // batch_size
#F_rgb_model.fit_generator(train_batch_generator, epochs=10, steps_per_epoch=472//1, verbose=1) #train_length // batch_size loss = 0.4

In [ ]:
#F_rgb_model.save_weights("64_rgb_model_2.h5")
#!cp '64_rgb_model_2.h5' '/content/drive/My Drive/cv/'


test on rgb(best results)
---

In [26]:
import numpy as np
import argparse

from i3d_inception import Inception_Inflated3d

NUM_CLASSES = 5
FRAME_HEIGHT = FRAME_WIDTH = 224
NUM_RGB_CHANNELS = 3
NUM_FLOW_CHANNELS = 2
NUM_FRAMES = 64

#define models
# build model for rgb data
# and load pretrained weights (trained on imagenet and kinetics dataset)
rgb_model = Inception_Inflated3d(
                include_top=False,
                weights=None,
                input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS),
                classes=NUM_CLASSES)

x = rgb_model.get_layer('global_avg_pool').output

x = Dropout(0.0)(x)

x = conv3d_bn(x, NUM_CLASSES, 1, 1, 1, padding='same', 
        use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')
 
num_frames_remaining = int(x.shape[1])
x = Reshape((num_frames_remaining, NUM_CLASSES))(x)

# logits (raw scores for each class)
x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
            output_shape=lambda s: (s[0], s[2]))(x)

x = Activation('softmax', name='prediction')(x)

rgb_model = Model(rgb_model.get_layer(index = 0).output, x, name='i3d_inception_rgb')

rgb_model.load_weights("/content/drive/My Drive/cv/a_rgb_model.h5")
#rgb_model.load_weights("/content/drive/My Drive/a_rgb_model.h5")


In [27]:
#sort test vids
vid_paths = ["" for x in range(126)]

i = 0
for video in os.listdir(test_path):
    start = video.find('(')
    end = video.find(')')

    num = video[start+1:end]
    num = int(num)
    vid_paths[num-1] = video
    i+=1

print(vid_paths)

['test_image (1).mpg', 'test_image (2).mpg', 'test_image (3).mpg', 'test_image (4).mpg', 'test_image (5).mpg', 'test_image (6).mpg', 'test_image (7).mpg', 'test_image (8).mpg', 'test_image (9).mpg', 'test_image (10).mpg', 'test_image (11).mpg', 'test_image (12).mpg', 'test_image (13).mpg', 'test_image (14).mpg', 'test_image (15).mpg', 'test_image (16).mpg', 'test_image (17).mpg', 'test_image (18).mpg', 'test_image (19).mpg', 'test_image (20).mpg', 'test_image (21).mpg', 'test_image (22).mpg', 'test_image (23).mpg', 'test_image (24).mpg', 'test_image (25).mpg', 'test_image (26).mpg', 'test_image (27).mpg', 'test_image (28).mpg', 'test_image (29).mpg', 'test_image (30).mpg', 'test_image (31).mpg', 'test_image (32).mpg', 'test_image (33).mpg', 'test_image (34).mpg', 'test_image (35).mpg', 'test_image (36).mpg', 'test_image (37).mpg', 'test_image (38).mpg', 'test_image (39).mpg', 'test_image (40).mpg', 'test_image (41).mpg', 'test_image (42).mpg', 'test_image (43).mpg', 'test_image (44).mp

In [29]:
clss = {1:0, 2:1, 0:2, 3:3, 4:4}

In [31]:
#                 0           1          2        3          4
#            Diving         Jumping    Basketball  Tennis   Walking
#                1            2          0           3        4
i = 0
acc = 0
pred = []
for video in vid_paths:
    flow = extract_rgb(test_path+video, NUM_FRAMES)
    flow_logits = rgb_model.predict(flow)
    flow_logits = flow_logits[0]

    sample_predictions = np.exp(flow_logits) / np.sum(np.exp(flow_logits))

    sorted_indices = np.argsort(sample_predictions)[::-1]
    print(clss[sorted_indices[0]])
    pred.append(clss[sorted_indices[0]])
    i+=1

print(pred)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
1
1
4
0
1
1
1
3
1
1
1
1
1
0
0
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 4, 0, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [ ]:
#write in file
file = open('submit.csv', '+w')
file.write('Id'+ ','+ 'Category' + '\n')
for i in range(len(pred)):
    row = str(vid_paths[i]) + ',' + str(pred[i]) + '\n'
    file.write(row)
file.close() 


flow model(don't run)
---

In [ ]:
# build model for optical flow data
# and load pretrained weights (trained on imagenet and kinetics dataset)
flow_model = Inception_Inflated3d(
                include_top=False,
                weights='flow_imagenet_and_kinetics',
                input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_FLOW_CHANNELS),
                classes=NUM_CLASSES)

/content/keras-kinetics-i3d/i3d_inception.py:101: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  str(input_shape[-1]) + ' input channels.')


49512448/49507528 [==============================] - 2s 0us/step


In [ ]:
x = flow_model.get_layer('global_avg_pool').output

x = Dropout(0.0)(x)

x = conv3d_bn(x, NUM_CLASSES, 1, 1, 1, padding='same', 
        use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')
 
num_frames_remaining = int(x.shape[1])
x = Reshape((num_frames_remaining, NUM_CLASSES))(x)

# logits (raw scores for each class)
x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
            output_shape=lambda s: (s[0], s[2]))(x)

x = Activation('softmax', name='prediction')(x)

F_flow_model = Model(flow_model.get_layer(index = 0).output, x, name='i3d_inception_flow')

In [ ]:
from random import shuffle
shuffle(train_paths)
train = train_paths[0:]
val = train_paths[337:]
print(len(train))
print(len(val))

global train_index
train_index = 0
global test_index
test_index = 0

def load_data(batch_size, paths, train_bool):
    
    index = 0
    if train_bool:
        global train_index
        if train_index == len(paths):
            train_index = 0
        index = train_index
    else:
        global test_index
        if test_index == len(paths):
            test_index = 0
        index = test_index

    example = paths[index]
    f = extract_flow(example[0], NUM_FRAMES)
    
    if train_bool:
        train_index+=1
    else:
        test_index+=1
    
    return (f, [example[1]])

def batch_generator(batch_size, steps, paths, train_bool):
    
    idx=1
    while True: 
        yield load_data(batch_size, paths, train_bool)## Yields data
        if idx<steps:
            idx+=1
        else:
            idx=1
train_batch_generator = batch_generator(1, 1, train, True)
val_batch_generator = batch_generator(1, 1, val, False)

472
135


In [ ]:
#F_flow_model.load_weights("a_flow_model.h5")

In [ ]:
F_flow_model.compile(optimizer=Adam(0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#F_flow_model.fit_generator(train_batch_generator, epochs=100, steps_per_epoch=377//1, verbose=1, validation_data=val_batch_generator, validation_steps=135//1) #train_length // batch_size
#F_flow_model.fit_generator(train_batch_generator, epochs=100, steps_per_epoch=472//1, verbose=1) #train_length // batch_size

Epoch 1/100
472/472 [==============================] - 976s 2s/step - loss: 1.5933 - accuracy: 0.3199
Epoch 2/100
472/472 [==============================] - 953s 2s/step - loss: 1.4643 - accuracy: 0.5275
Epoch 3/100
472/472 [==============================] - 941s 2s/step - loss: 1.3473 - accuracy: 0.6843
Epoch 4/100
472/472 [==============================] - 945s 2s/step - loss: 1.2303 - accuracy: 0.8008
Epoch 5/100
472/472 [==============================] - 947s 2s/step - loss: 1.1132 - accuracy: 0.8729
Epoch 6/100
472/472 [==============================] - 942s 2s/step - loss: 0.9968 - accuracy: 0.9216
Epoch 7/100
472/472 [==============================] - 973s 2s/step - loss: 0.8830 - accuracy: 0.9492
Epoch 8/100
472/472 [==============================] - 1029s 2s/step - loss: 0.7737 - accuracy: 0.9725
Epoch 9/100
472/472 [==============================] - 1032s 2s/step - loss: 0.6704 - accuracy: 0.9873
Epoch 10/100
472/472 [==============================] - 1042s 2s/step - loss: 0.

KeyboardInterrupt: ignored

In [ ]:
#F_flow_model.save_weights("a_flow_model.h5")
#!cp 'a_flow_model.h5' '/content/drive/My Drive/cv/'

test on flow

In [ ]:
#sort test vids
vid_paths = ["" for x in range(126)]

i = 0
for video in os.listdir(test_path):
    start = video.find('(')
    end = video.find(')')

    num = video[start+1:end]
    num = int(num)
    vid_paths[num-1] = video
    i+=1

print(vid_paths)

['test_image (1).mpg', 'test_image (2).mpg', 'test_image (3).mpg', 'test_image (4).mpg', 'test_image (5).mpg', 'test_image (6).mpg', 'test_image (7).mpg', 'test_image (8).mpg', 'test_image (9).mpg', 'test_image (10).mpg', 'test_image (11).mpg', 'test_image (12).mpg', 'test_image (13).mpg', 'test_image (14).mpg', 'test_image (15).mpg', 'test_image (16).mpg', 'test_image (17).mpg', 'test_image (18).mpg', 'test_image (19).mpg', 'test_image (20).mpg', 'test_image (21).mpg', 'test_image (22).mpg', 'test_image (23).mpg', 'test_image (24).mpg', 'test_image (25).mpg', 'test_image (26).mpg', 'test_image (27).mpg', 'test_image (28).mpg', 'test_image (29).mpg', 'test_image (30).mpg', 'test_image (31).mpg', 'test_image (32).mpg', 'test_image (33).mpg', 'test_image (34).mpg', 'test_image (35).mpg', 'test_image (36).mpg', 'test_image (37).mpg', 'test_image (38).mpg', 'test_image (39).mpg', 'test_image (40).mpg', 'test_image (41).mpg', 'test_image (42).mpg', 'test_image (43).mpg', 'test_image (44).mp

In [ ]:

i = 0
acc = 0
pred = []
for video in vid_paths:
    flow = extract_flow(test_path+video, NUM_FRAMES)
    flow_logits = F_flow_model.predict(flow)
    flow_logits = flow_logits[0]

    sample_predictions = np.exp(flow_logits) / np.sum(np.exp(flow_logits))

    sorted_indices = np.argsort(sample_predictions)[::-1]

    pred.append(sorted_indices[0])
    i+=1


print(pred)

In [ ]:
#write in file
file = open('submit_flow.csv', '+w')
file.write('Id'+ ','+ 'Category' + '\n')
for i in range(len(pred)):
    row = str(vid_paths[i]) + ',' + str(pred[i]) + '\n'
    file.write(row)
file.close() 


testing on both together(don't run)
---

In [ ]:
NUM_CLASSES = 5
FRAME_HEIGHT = FRAME_WIDTH = 224
NUM_RGB_CHANNELS = 3
NUM_FLOW_CHANNELS = 2
NUM_FRAMES = 125

In [ ]:
#define models
# build model for rgb data
# and load pretrained weights (trained on imagenet and kinetics dataset)
t_rgb_model = Inception_Inflated3d(
                include_top=False,
                weights=None,
                input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS),
                classes=NUM_CLASSES)
 
# build model for optical flow data
# and load pretrained weights (trained on imagenet and kinetics dataset)
t_flow_model = Inception_Inflated3d(
                include_top=False,
                weights=None,
                input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_FLOW_CHANNELS),
                classes=NUM_CLASSES)

/content/keras-kinetics-i3d/i3d_inception.py:101: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  str(input_shape[-1]) + ' input channels.')


In [ ]:
x = t_rgb_model.get_layer('global_avg_pool').output

x = Dropout(0.0)(x)

x = conv3d_bn(x, NUM_CLASSES, 1, 1, 1, padding='same', 
        use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')
 
num_frames_remaining = int(x.shape[1])
x = Reshape((num_frames_remaining, NUM_CLASSES))(x)

# logits (raw scores for each class)
x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
            output_shape=lambda s: (s[0], s[2]))(x)

x = Activation('softmax', name='prediction')(x)

F_t_rgb_model = Model(t_rgb_model.get_layer(index = 0).output, x, name='i3d_inception_rgb_test')

########################################

x = t_flow_model.get_layer('global_avg_pool').output

x = Dropout(0.0)(x)

x = conv3d_bn(x, NUM_CLASSES, 1, 1, 1, padding='same', 
        use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')
 
num_frames_remaining = int(x.shape[1])
x = Reshape((num_frames_remaining, NUM_CLASSES))(x)

# logits (raw scores for each class)
x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
            output_shape=lambda s: (s[0], s[2]))(x)

x = Activation('softmax', name='prediction')(x)

F_t_flow_model = Model(t_flow_model.get_layer(index = 0).output, x, name='i3d_inception_flow_test')

In [ ]:
F_t_rgb_model.load_weights('/content/drive/My Drive/cv/90_rgb_model.h5')
F_t_flow_model.load_weights('/content/drive/My Drive/cv/90_flow_model.h5')

In [ ]:
#sort test vids
vid_paths = ["" for x in range(126)]

i = 0
for video in os.listdir(test_path):
    start = video.find('(')
    end = video.find(')')

    num = video[start+1:end]
    num = int(num)
    vid_paths[num-1] = video
    i+=1

print(vid_paths)


['test_image (1).mpg', 'test_image (2).mpg', 'test_image (3).mpg', 'test_image (4).mpg', 'test_image (5).mpg', 'test_image (6).mpg', 'test_image (7).mpg', 'test_image (8).mpg', 'test_image (9).mpg', 'test_image (10).mpg', 'test_image (11).mpg', 'test_image (12).mpg', 'test_image (13).mpg', 'test_image (14).mpg', 'test_image (15).mpg', 'test_image (16).mpg', 'test_image (17).mpg', 'test_image (18).mpg', 'test_image (19).mpg', 'test_image (20).mpg', 'test_image (21).mpg', 'test_image (22).mpg', 'test_image (23).mpg', 'test_image (24).mpg', 'test_image (25).mpg', 'test_image (26).mpg', 'test_image (27).mpg', 'test_image (28).mpg', 'test_image (29).mpg', 'test_image (30).mpg', 'test_image (31).mpg', 'test_image (32).mpg', 'test_image (33).mpg', 'test_image (34).mpg', 'test_image (35).mpg', 'test_image (36).mpg', 'test_image (37).mpg', 'test_image (38).mpg', 'test_image (39).mpg', 'test_image (40).mpg', 'test_image (41).mpg', 'test_image (42).mpg', 'test_image (43).mpg', 'test_image (44).mp

In [ ]:
i = 0

pred = []
for video in vid_paths:
    print(video)
    
    rgb = extract_rgb(test_path+video, NUM_FRAMES)
    flow = extract_flow(test_path+video, NUM_FRAMES)
    
    rgb_logits = F_t_rgb_model.predict(rgb)
    flow_logits = F_t_flow_model.predict(flow)
    sample_logits = rgb_logits + flow_logits

    sample_logits = sample_logits[0] # we are dealing with just one example
    sample_predictions = np.exp(sample_logits) / np.sum(np.exp(sample_logits))

    sorted_indices = np.argsort(sample_predictions)[::-1]
    pred.append(sorted_indices[0])


    i+=1


In [ ]:
#write in file
file = open('submit_both.csv', '+w')

file.write('Id'+ ','+ 'Category' + '\n')
for i in range(len(pred)):
    row = str(vid_paths[i]) + ',' + str(pred[i]) + '\n'
    file.write(row)
    
file.close() 